In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import string
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.porter import PorterStemmer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional 
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
basketball = pd.read_csv("/content/drive/My Drive/Tolu/tolufilebasketball.csv")


tennis = pd.read_csv("/content/drive/My Drive/Tolu/tolufiletennis.csv")

cricket = pd.read_csv("/content/drive/My Drive/Tolu/tolufilecricket.csv")

In [ ]:
headlines = pd.concat([basketball, tennis, cricket], axis=0)

In [ ]:
headlines = headlines.sample(frac = 1)

In [ ]:
headlines

,title,value
2100,AP Interview: Tennis tour CEO still wants Peng...,0
3969,Colorado tops UC Riverside 82-66 behind Hadley...,1
1826,Key's 17 leads No. 11 Volunteers over Tennesse...,1
3707,"McDermott, Johnson lead Spurs to second victor...",1
2334,Semi final World Cup 2022 schedule and fixture...,2
...,...,...
443,Pakistan call up Haris after knee injury sidel...,2
916,Amazon Prime Video Expands Basketball Programm...,1
326,"Canines, anyone? Westminster dog show heads to...",0
1653,"Mavs bounce back, end Banchero's 20-point run ...",1


In [ ]:
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", str(text))


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)
def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)
  

In [ ]:
headlines["title"] = headlines.title.map(lambda x: remove_URL(x))
headlines["title"] = headlines.title.map(lambda x: remove_html(x))
headlines["title"] = headlines.title.map(lambda x: remove_emoji(x))
headlines["title"] = headlines.title.map(lambda x: remove_punct(x))

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)
headlines["title"] =headlines["title"].map(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
headlines

,title,value
2100,ap interview tennis tour ceo still wants peng ...,0
3969,colorado tops uc riverside 8266 behind hadley ...,1
1826,keys 17 leads 11 volunteers tennessee tech 7543,1
3707,mcdermott johnson lead spurs second victory wo...,1
2334,semi final world cup 2022 schedule fixtures t2...,2
...,...,...
443,pakistan call haris knee injury sidelines zaman,2
916,amazon prime video expands basketball programm...,1
326,canines anyone westminster dog show heads tenn...,0
1653,mavs bounce back end bancheros 20point run bea...,1


In [ ]:
vocabulary_size = 10000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(headlines['title'])
sequences = tokenizer.texts_to_sequences(headlines['title'])
X_LSTM = pad_sequences(sequences, maxlen=50)

In [ ]:
## Split the data into train and test
Y_LSTM = headlines["value"]
X_train_LSTM, X_test_LSTM, Y_train_LSTM, Y_test_LSTM = train_test_split(X_LSTM, 
                       Y_LSTM, test_size=0.3, random_state=7)

In [ ]:
X_test_LSTM.shape

(3000, 50)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
def create_model(input_length=50):
   model = Sequential()
   model.add(Embedding(10000, 300, input_length=50)) 
   model.add(Bidirectional(LSTM(100)))
   model.add(Dense(1,activation='sigmoid'))
   model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  
   return model    
model_LSTM = KerasClassifier(build_fn=create_model, epochs=3, verbose=1, validation_split=0.4)
model_LSTM.fit(X_train_LSTM, Y_train_LSTM)

<ipython-input-15-9f996c172b6c>:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_LSTM = KerasClassifier(build_fn=create_model, epochs=3, verbose=1, validation_split=0.4)


Epoch 1/3
132/132 [==============================] - 21s 138ms/step - loss: 0.4485 - accuracy: 0.8193 - val_loss: 0.1518 - val_accuracy: 0.9432
Epoch 2/3
132/132 [==============================] - 17s 131ms/step - loss: 0.0576 - accuracy: 0.9810 - val_loss: 0.1408 - val_accuracy: 0.9486
Epoch 3/3
132/132 [==============================] - 17s 132ms/step - loss: 0.0246 - accuracy: 0.9917 - val_loss: 0.1370 - val_accuracy: 0.9518


In [ ]:
# performance metrics
import sklearn
from sklearn.metrics import classification_report
y_pred = model_LSTM.predict(X_test_LSTM)
report = classification_report(Y_test_LSTM, y_pred.round())
print(report)

94/94 [==============================] - 3s 29ms/step
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       765
           1       0.97      0.97      0.97      1490
           2       0.94      0.92      0.93       745

    accuracy                           0.94      3000
   macro avg       0.94      0.94      0.94      3000
weighted avg       0.94      0.94      0.94      3000

